In [1]:
#import scipy
#rate,data = scipy.io.wavfile.read('./vocalSeparation/origin_mix.wav')
#scipy.io.wavfile.write('./vocalSeparation/morigin_mix.wav',rate,data)

In [2]:
#import soundfile as sf
#data, samplerate = sf.read('./vocalSeparation/pred_mix.wav', dtype='float32')
#data = librosa.resample(data.T, samplerate, 16000)

In [3]:
"""Training script for the WaveNet network on the VCTK corpus.

This script trains a network with the WaveNet using data from the VCTK corpus,
which can be freely downloaded at the following site (~10 GB):
http://homepages.inf.ed.ac.uk/jyamagis/page3/page58/page58.html
"""

from __future__ import print_function

import argparse
from datetime import datetime
import json
import os
import sys
import time
import numpy as np
import tensorflow as tf
from tensorflow.python.client import timeline
import soundfile as sf

from wavenetVS import WaveNetModel, AudioReader, optimizer_factory


'''import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"'''


'import os\nos.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"\nos.environ["CUDA_VISIBLE_DEVICES"] = "0"'

In [4]:
BATCH_SIZE = 1
DATA_DIRECTORY = './vsCorpus'
LOGDIR_ROOT = './logdirVS'
CHECKPOINT_EVERY = 50
NUM_STEPS = int(1e5)
LEARNING_RATE = 1e-3
#L2_REGULARIZATION_STRENGTH = 1e-5
L2_REGULARIZATION_STRENGTH = 0
WAVENET_PARAMS = './wavenet_params.json'
STARTED_DATESTRING = "{0:%Y-%m-%dT%H-%M-%S}".format(datetime.now())
SAMPLE_SIZE = 100000
#SILENCE_THRESHOLD = 0.3
SILENCE_THRESHOLD = 0
EPSILON = 0.001
MOMENTUM = 0.9
MAX_TO_KEEP = 5
METADATA = False

In [5]:
def get_arguments():
    def _str_to_bool(s):
        """Convert string to bool (in argparse context)."""
        if s.lower() not in ['true', 'false']:
            raise ValueError('Argument needs to be a '
                             'boolean, got {}'.format(s))
        return {'true': True, 'false': False}[s.lower()]

    parser = argparse.ArgumentParser(description='WaveNet example network')
    parser.add_argument('--batch_size', type=int, default=BATCH_SIZE,
                        help='How many wav files to process at once. Default: ' + str(BATCH_SIZE) + '.')
    parser.add_argument('--data_dir', type=str, default=DATA_DIRECTORY,
                        help='The directory containing the VCTK corpus.')
    parser.add_argument('--store_metadata', type=bool, default=METADATA,
                        help='Whether to store advanced debugging information '
                        '(execution time, memory consumption) for use with '
                        'TensorBoard. Default: ' + str(METADATA) + '.')
    parser.add_argument('--logdir', type=str, default=None,
                        help='Directory in which to store the logging '
                        'information for TensorBoard. '
                        'If the model already exists, it will restore '
                        'the state and will continue training. '
                        'Cannot use with --logdir_root and --restore_from.')
    parser.add_argument('--logdir_root', type=str, default=None,
                        help='Root directory to place the logging '
                        'output and generated model. These are stored '
                        'under the dated subdirectory of --logdir_root. '
                        'Cannot use with --logdir.')
    parser.add_argument('--restore_from', type=str, default=None,
                        help='Directory in which to restore the model from. '
                        'This creates the new model under the dated directory '
                        'in --logdir_root. '
                        'Cannot use with --logdir.')
    parser.add_argument('--checkpoint_every', type=int,
                        default=CHECKPOINT_EVERY,
                        help='How many steps to save each checkpoint after. Default: ' + str(CHECKPOINT_EVERY) + '.')
    parser.add_argument('--num_steps', type=int, default=NUM_STEPS,
                        help='Number of training steps. Default: ' + str(NUM_STEPS) + '.')
    parser.add_argument('--learning_rate', type=float, default=LEARNING_RATE,
                        help='Learning rate for training. Default: ' + str(LEARNING_RATE) + '.')
    parser.add_argument('--wavenet_params', type=str, default=WAVENET_PARAMS,
                        help='JSON file with the network parameters. Default: ' + WAVENET_PARAMS + '.')
    parser.add_argument('--sample_size', type=int, default=SAMPLE_SIZE,
                        help='Concatenate and cut audio samples to this many '
                        'samples. Default: ' + str(SAMPLE_SIZE) + '.')
    parser.add_argument('--l2_regularization_strength', type=float,
                        default=L2_REGULARIZATION_STRENGTH,
                        help='Coefficient in the L2 regularization. '
                        'Default: False')
    parser.add_argument('--silence_threshold', type=float,
                        default=SILENCE_THRESHOLD,
                        help='Volume threshold below which to trim the start '
                        'and the end from the training set samples. Default: ' + str(SILENCE_THRESHOLD) + '.')
    parser.add_argument('--optimizer', type=str, default='adam',
                        choices=optimizer_factory.keys(),
                        help='Select the optimizer specified by this option. Default: adam.')
    parser.add_argument('--momentum', type=float,
                        default=MOMENTUM, help='Specify the momentum to be '
                        'used by sgd or rmsprop optimizer. Ignored by the '
                        'adam optimizer. Default: ' + str(MOMENTUM) + '.')
    parser.add_argument('--histograms', type=_str_to_bool, default=False,
                        help='Whether to store histogram summaries. Default: False')
    parser.add_argument('--gc_channels', type=int, default=None,
                        help='Number of global condition channels. Default: None. Expecting: Int')
    parser.add_argument('--max_checkpoints', type=int, default=MAX_TO_KEEP,
                        help='Maximum amount of checkpoints that will be kept alive. Default: '
                             + str(MAX_TO_KEEP) + '.')
    return parser.parse_args([])


In [6]:
def save(saver, sess, logdir, step):
    model_name = 'model.ckpt'
    checkpoint_path = os.path.join(logdir, model_name)
    print('Storing checkpoint to {} ...'.format(logdir), end="")
    sys.stdout.flush()

    if not os.path.exists(logdir):
        os.makedirs(logdir)

    saver.save(sess, checkpoint_path, global_step=step)
    print(' Done.')

In [7]:
def load(saver, sess, logdir):
    print("Trying to restore saved checkpoints from {} ...".format(logdir),
          end="")

    ckpt = tf.train.get_checkpoint_state(logdir)
    if ckpt:
        print("  Checkpoint found: {}".format(ckpt.model_checkpoint_path))
        global_step = int(ckpt.model_checkpoint_path
                          .split('/')[-1]
                          .split('-')[-1])
        print("  Global step was: {}".format(global_step))
        print("  Restoring...", end="")
        saver.restore(sess, ckpt.model_checkpoint_path)
        print(" Done.")
        return global_step
    else:
        print(" No checkpoint found.")
        return None

In [ ]:
def get_default_logdir(logdir_root):
    logdir = os.path.join(logdir_root, 'train', STARTED_DATESTRING)
    return logdir


def validate_directories(args):
    """Validate and arrange directory related arguments."""

    # Validation
    if args.logdir and args.logdir_root:
        raise ValueError("--logdir and --logdir_root cannot be "
                         "specified at the same time.")

    if args.logdir and args.restore_from:
        raise ValueError(
            "--logdir and --restore_from cannot be specified at the same "
            "time. This is to keep your previous model from unexpected "
            "overwrites.\n"
            "Use --logdir_root to specify the root of the directory which "
            "will be automatically created with current date and time, or use "
            "only --logdir to just continue the training from the last "
            "checkpoint.")

    # Arrangement
    logdir_root = args.logdir_root
    if logdir_root is None:
        logdir_root = LOGDIR_ROOT

    logdir = args.logdir
    if logdir is None:
        logdir = get_default_logdir(logdir_root)
        print('Using default logdir: {}'.format(logdir))

    restore_from = args.restore_from
    if restore_from is None:
        # args.logdir and args.restore_from are exclusive,
        # so it is guaranteed the logdir here is newly created.
        restore_from = logdir

    return {
        'logdir': logdir,
        'logdir_root': args.logdir_root,
        'restore_from': restore_from
    }


In [ ]:
args = get_arguments()

try:
    directories = validate_directories(args)
except ValueError as e:
    print("Some arguments are wrong:")
    print(str(e))

logdir = directories['logdir']
restore_from = directories['restore_from']

# Even if we restored the model, we will treat it as new training
# if the trained model is written into an arbitrary location.
is_overwritten_training = logdir != restore_from

with open(args.wavenet_params, 'r') as f:
    wavenet_params = json.load(f)

# Create coordinator.
coord = tf.train.Coordinator()

# Load raw waveform from VCTK corpus.
with tf.name_scope('create_inputs'):
    # Allow silence trimming to be skipped by specifying a threshold near
    # zero.
    silence_threshold = args.silence_threshold if args.silence_threshold > \
                                                  EPSILON else None
    gc_enabled = args.gc_channels is not None
    reader = AudioReader(
        args.data_dir,
        coord,
        sample_rate=wavenet_params['sample_rate'],   #"sample_rate": 16000,
        gc_enabled=gc_enabled,
        receptive_field=WaveNetModel.calculate_receptive_field(wavenet_params["filter_width"],
                                                               wavenet_params["dilations"],
                                                               wavenet_params["scalar_input"],
                                                               wavenet_params["initial_filter_width"]),
        sample_size=args.sample_size,  #SAMPLE_SIZE = 100000
        silence_threshold=silence_threshold)
    traudio_batch = reader.trdequeue(args.batch_size)  #BATCH_SIZE = 1
    if gc_enabled:
        ##TODO train and val
        gc_id_batch = reader.dequeue_gc(args.batch_size)
    else:
        gc_id_batch = None

# Create network.
net = WaveNetModel(
    batch_size=args.batch_size,
    dilations=wavenet_params["dilations"],
    filter_width=wavenet_params["filter_width"],
    residual_channels=wavenet_params["residual_channels"],
    dilation_channels=wavenet_params["dilation_channels"],
    skip_channels=wavenet_params["skip_channels"],
    quantization_channels=wavenet_params["quantization_channels"],
    use_biases=wavenet_params["use_biases"],
    scalar_input=wavenet_params["scalar_input"],
    initial_filter_width=wavenet_params["initial_filter_width"],
    histograms=args.histograms,
    global_condition_channels=args.gc_channels,
    global_condition_cardinality=reader.gc_category_cardinality)

if args.l2_regularization_strength == 0:
    args.l2_regularization_strength = None
trloss = net.trloss(input_batch=traudio_batch,
                global_condition_batch=gc_id_batch,
                l2_regularization_strength=args.l2_regularization_strength)
optimizer = optimizer_factory[args.optimizer](
                learning_rate=args.learning_rate,
                momentum=args.momentum)
trainable = tf.trainable_variables()
optim = optimizer.minimize(trloss, var_list=trainable)

# Set up logging for TensorBoard.
writer = tf.summary.FileWriter(logdir)
writer.add_graph(tf.get_default_graph())
run_metadata = tf.RunMetadata()
summaries = tf.summary.merge_all()

# Set up session
config = tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))

vaudio_batch = reader.valbatch()
valloss = net.valloss(input_batch=vaudio_batch,
                global_condition_batch=gc_id_batch,
                l2_regularization_strength=args.l2_regularization_strength)
genfile = net.generateFile(input_batch=vaudio_batch,
                global_condition_batch=gc_id_batch,
                l2_regularization_strength=args.l2_regularization_strength)

init = tf.global_variables_initializer()
sess.run(init)

# Saver for storing checkpoints of the model.
saver = tf.train.Saver(var_list=tf.trainable_variables(), max_to_keep=args.max_checkpoints)

try:
    saved_global_step = load(saver, sess, restore_from)
    if is_overwritten_training or saved_global_step is None:
        # The first training step will be saved_global_step + 1,
        # therefore we put -1 here for new or overwritten trainings.
        saved_global_step = -1

except:
    print("Something went wrong while restoring checkpoint. "
          "We will terminate training to avoid accidentally overwriting "
          "the previous model.")
    raise

    
    
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
reader.start_threads(sess)


step = None
last_saved_step = saved_global_step
minvalloss = 10000
try:
    for step in range(saved_global_step + 1, args.num_steps):
        start_time = time.time()
        if args.store_metadata and step % 50 == 0:
            # Slow run that stores extra information for debugging.
            print('Storing metadata')
            run_options = tf.RunOptions(
                trace_level=tf.RunOptions.FULL_TRACE)
            summary, trloss_value, _ = sess.run(
                [summaries, trloss, optim],
                options=run_options,
                run_metadata=run_metadata)
            writer.add_summary(summary, step)
            writer.add_run_metadata(run_metadata,
                                    'step_{:04d}'.format(step))
            tl = timeline.Timeline(run_metadata.step_stats)
            timeline_path = os.path.join(logdir, 'timeline.trace')
            with open(timeline_path, 'w') as f:
                f.write(tl.generate_chrome_trace_format(show_memory=True))
        else:
            summary, trloss_value, _ = sess.run([summaries, trloss, optim])
            writer.add_summary(summary, step)
        duration = time.time() - start_time
        print('step {:d} - trloss = {:.3f}, ({:.3f} sec/step)'
              .format(step, trloss_value, duration))
        
        
        if step % args.checkpoint_every == 0:
            if step != 0:    
                ans = sess.run(genfile)
                sf.write('./vsCorpus/ans/'+str(step)+'.wav',ans.reshape(-1),16000)
                print('stored done')
            valloss_value = sess.run(valloss)
            print('validateLoss = {:.3f}, ({:.3f} sec/step)'
              .format(valloss_value, time.time() - start_time))
            #if(valloss_value < minvalloss):
                #save(saver, sess, logdir, step)
                #last_saved_step = step
                #minvalloss = valloss_value

except KeyboardInterrupt:
    # Introduce a line break after ^C is displayed so save message
    # is on its own line.
    print()
finally:
    if step > last_saved_step:
        save(saver, sess, logdir, step)
    coord.request_stop()
    coord.join(threads)




Using default logdir: ./logdirVS/train/2018-06-06T13-52-42
trdequeue
raw_output Tensor("wavenet_2/postprocessing/Add_1:0", shape=(?, ?, 256), dtype=float32)
ans0 (?,)
Trying to restore saved checkpoints from ./logdirVS/train/2018-06-06T13-52-42 ... No checkpoint found.
step 0 - trloss = 5.667, (24.163 sec/step)
validateLoss = 5.631, (216.728 sec/step)
step 1 - trloss = 5.636, (2.636 sec/step)
step 2 - trloss = 5.609, (2.628 sec/step)
step 3 - trloss = 5.584, (2.630 sec/step)
step 4 - trloss = 5.552, (2.632 sec/step)
step 5 - trloss = 5.535, (2.632 sec/step)
step 6 - trloss = 5.516, (2.630 sec/step)
step 7 - trloss = 5.556, (2.631 sec/step)
step 8 - trloss = 5.478, (2.634 sec/step)
step 9 - trloss = 5.488, (2.633 sec/step)
step 10 - trloss = 5.490, (2.636 sec/step)
step 11 - trloss = 5.501, (2.636 sec/step)
step 12 - trloss = 5.461, (2.635 sec/step)
step 13 - trloss = 5.486, (2.641 sec/step)
step 14 - trloss = 5.574, (2.640 sec/step)
step 15 - trloss = 5.476, (2.636 sec/step)
step 16 - 

step 179 - trloss = 4.844, (2.643 sec/step)
step 180 - trloss = 4.905, (2.643 sec/step)
step 181 - trloss = 4.996, (2.643 sec/step)
step 182 - trloss = 4.849, (2.642 sec/step)
step 183 - trloss = 4.862, (2.643 sec/step)
step 184 - trloss = 4.834, (2.642 sec/step)
step 185 - trloss = 4.864, (2.643 sec/step)
step 186 - trloss = 4.853, (2.642 sec/step)
step 187 - trloss = 4.890, (2.642 sec/step)
step 188 - trloss = 4.778, (2.643 sec/step)
step 189 - trloss = 4.801, (2.641 sec/step)
step 190 - trloss = 4.734, (2.642 sec/step)
step 191 - trloss = 4.799, (2.642 sec/step)
step 192 - trloss = 4.822, (2.643 sec/step)
step 193 - trloss = 4.879, (2.643 sec/step)
step 194 - trloss = 4.794, (2.643 sec/step)
step 195 - trloss = 4.847, (2.642 sec/step)
step 196 - trloss = 4.811, (2.645 sec/step)
step 197 - trloss = 4.836, (2.644 sec/step)
step 198 - trloss = 4.694, (2.645 sec/step)
step 199 - trloss = 4.784, (2.644 sec/step)
step 200 - trloss = 4.833, (2.645 sec/step)
stored done
validateLoss = 4.841

step 361 - trloss = 4.685, (2.644 sec/step)
step 362 - trloss = 4.689, (2.643 sec/step)
step 363 - trloss = 4.634, (2.643 sec/step)
step 364 - trloss = 4.611, (2.640 sec/step)
step 365 - trloss = 4.532, (2.642 sec/step)
step 366 - trloss = 4.736, (2.642 sec/step)
step 367 - trloss = 4.556, (2.642 sec/step)
step 368 - trloss = 4.645, (2.642 sec/step)
step 369 - trloss = 4.658, (2.643 sec/step)
step 370 - trloss = 4.585, (2.643 sec/step)
step 371 - trloss = 4.635, (2.642 sec/step)
step 372 - trloss = 4.519, (2.644 sec/step)
step 373 - trloss = 4.639, (2.643 sec/step)
step 374 - trloss = 4.695, (2.642 sec/step)
step 375 - trloss = 4.626, (2.643 sec/step)
step 376 - trloss = 4.671, (2.642 sec/step)
step 377 - trloss = 4.663, (2.642 sec/step)
step 378 - trloss = 4.688, (2.643 sec/step)
step 379 - trloss = 4.740, (2.643 sec/step)
step 380 - trloss = 4.771, (2.641 sec/step)
step 381 - trloss = 4.611, (2.643 sec/step)
step 382 - trloss = 4.707, (2.642 sec/step)
step 383 - trloss = 4.612, (2.64

step 544 - trloss = 4.644, (2.645 sec/step)
step 545 - trloss = 4.566, (2.645 sec/step)
step 546 - trloss = 4.645, (2.643 sec/step)
step 547 - trloss = 4.606, (2.646 sec/step)
step 548 - trloss = 4.580, (2.645 sec/step)
step 549 - trloss = 4.677, (2.644 sec/step)
step 550 - trloss = 4.612, (2.645 sec/step)
stored done
validateLoss = 4.796, (6.862 sec/step)
step 551 - trloss = 4.639, (2.646 sec/step)
step 552 - trloss = 4.584, (2.645 sec/step)
step 553 - trloss = 4.540, (2.648 sec/step)
step 554 - trloss = 4.633, (2.645 sec/step)
step 555 - trloss = 4.690, (2.644 sec/step)
step 556 - trloss = 4.605, (2.645 sec/step)
step 557 - trloss = 4.578, (2.645 sec/step)
step 558 - trloss = 4.529, (2.647 sec/step)
step 559 - trloss = 4.604, (2.644 sec/step)
step 560 - trloss = 4.519, (2.649 sec/step)
step 561 - trloss = 4.503, (2.645 sec/step)
step 562 - trloss = 4.628, (2.644 sec/step)
step 563 - trloss = 4.553, (2.643 sec/step)
step 564 - trloss = 4.700, (2.644 sec/step)
step 565 - trloss = 4.605

step 726 - trloss = 4.465, (2.645 sec/step)
step 727 - trloss = 4.557, (2.646 sec/step)
step 728 - trloss = 4.600, (2.645 sec/step)
step 729 - trloss = 4.600, (2.648 sec/step)
step 730 - trloss = 4.453, (2.646 sec/step)
step 731 - trloss = 4.539, (2.643 sec/step)
step 732 - trloss = 4.499, (2.643 sec/step)
step 733 - trloss = 4.522, (2.645 sec/step)
step 734 - trloss = 4.411, (2.645 sec/step)
step 735 - trloss = 4.567, (2.645 sec/step)
step 736 - trloss = 4.611, (2.645 sec/step)
step 737 - trloss = 4.442, (2.644 sec/step)
step 738 - trloss = 4.463, (2.645 sec/step)
step 739 - trloss = 4.563, (2.644 sec/step)
step 740 - trloss = 4.591, (2.644 sec/step)
step 741 - trloss = 4.429, (2.643 sec/step)
step 742 - trloss = 4.513, (2.642 sec/step)
step 743 - trloss = 4.512, (2.643 sec/step)
step 744 - trloss = 4.540, (2.642 sec/step)
step 745 - trloss = 4.514, (2.642 sec/step)
step 746 - trloss = 4.604, (2.641 sec/step)
step 747 - trloss = 4.543, (2.640 sec/step)
step 748 - trloss = 4.590, (2.64

...

step 11057 - trloss = 3.835, (2.643 sec/step)
step 11058 - trloss = 3.832, (2.639 sec/step)
step 11059 - trloss = 3.633, (2.640 sec/step)
step 11060 - trloss = 3.622, (2.640 sec/step)
step 11061 - trloss = 3.880, (2.641 sec/step)
step 11062 - trloss = 3.829, (2.638 sec/step)
step 11063 - trloss = 3.938, (2.639 sec/step)
step 11064 - trloss = 3.500, (2.639 sec/step)
step 11065 - trloss = 3.893, (2.639 sec/step)
step 11066 - trloss = 3.670, (2.639 sec/step)
step 11067 - trloss = 3.720, (2.638 sec/step)
step 11068 - trloss = 3.627, (2.639 sec/step)
step 11069 - trloss = 3.809, (2.640 sec/step)
step 11070 - trloss = 3.858, (2.639 sec/step)
step 11071 - trloss = 3.771, (2.638 sec/step)
step 11072 - trloss = 3.786, (2.639 sec/step)
step 11073 - trloss = 3.898, (2.639 sec/step)
step 11074 - trloss = 3.887, (2.639 sec/step)
step 11075 - trloss = 3.549, (2.638 sec/step)
step 11076 - trloss = 3.824, (2.638 sec/step)
step 11077 - trloss = 3.597, (2.641 sec/step)
step 11078 - trloss = 3.889, (2.63

step 11232 - trloss = 3.677, (2.642 sec/step)
step 11233 - trloss = 3.502, (2.641 sec/step)
step 11234 - trloss = 3.754, (2.640 sec/step)
step 11235 - trloss = 3.785, (2.639 sec/step)
step 11236 - trloss = 3.783, (2.640 sec/step)
step 11237 - trloss = 3.633, (2.640 sec/step)
step 11238 - trloss = 3.561, (2.641 sec/step)
step 11239 - trloss = 3.647, (2.640 sec/step)
step 11240 - trloss = 3.802, (2.640 sec/step)
step 11241 - trloss = 3.635, (2.642 sec/step)
step 11242 - trloss = 3.545, (2.640 sec/step)
step 11243 - trloss = 3.754, (2.642 sec/step)
step 11244 - trloss = 3.605, (2.641 sec/step)
step 11245 - trloss = 3.557, (2.642 sec/step)
step 11246 - trloss = 3.522, (2.641 sec/step)
step 11247 - trloss = 3.787, (2.643 sec/step)
step 11248 - trloss = 3.754, (2.641 sec/step)
step 11249 - trloss = 3.540, (2.641 sec/step)
step 11250 - trloss = 3.747, (2.641 sec/step)
stored done
validateLoss = 6.172, (6.855 sec/step)
step 11251 - trloss = 3.578, (2.642 sec/step)
step 11252 - trloss = 3.585, 

step 11406 - trloss = 3.660, (2.641 sec/step)
step 11407 - trloss = 3.662, (2.641 sec/step)
step 11408 - trloss = 3.821, (2.641 sec/step)
step 11409 - trloss = 3.803, (2.641 sec/step)
step 11410 - trloss = 3.853, (2.641 sec/step)
step 11411 - trloss = 3.821, (2.641 sec/step)
step 11412 - trloss = 3.859, (2.640 sec/step)
step 11413 - trloss = 3.959, (2.640 sec/step)
step 11414 - trloss = 3.718, (2.640 sec/step)
step 11415 - trloss = 3.491, (2.641 sec/step)
step 11416 - trloss = 3.514, (2.641 sec/step)
step 11417 - trloss = 3.825, (2.640 sec/step)
step 11418 - trloss = 3.709, (2.640 sec/step)
step 11419 - trloss = 3.585, (2.642 sec/step)
step 11420 - trloss = 3.842, (2.641 sec/step)
step 11421 - trloss = 3.710, (2.642 sec/step)
step 11422 - trloss = 3.740, (2.642 sec/step)
step 11423 - trloss = 3.604, (2.640 sec/step)
step 11424 - trloss = 3.765, (2.641 sec/step)
step 11425 - trloss = 3.684, (2.642 sec/step)
step 11426 - trloss = 3.642, (2.642 sec/step)
step 11427 - trloss = 3.593, (2.64

step 11581 - trloss = 3.900, (2.642 sec/step)
step 11582 - trloss = 3.509, (2.641 sec/step)
step 11583 - trloss = 3.841, (2.639 sec/step)
step 11584 - trloss = 3.754, (2.640 sec/step)
step 11585 - trloss = 3.895, (2.643 sec/step)
step 11586 - trloss = 3.770, (2.643 sec/step)
step 11587 - trloss = 3.785, (2.641 sec/step)
step 11588 - trloss = 3.663, (2.643 sec/step)
step 11589 - trloss = 3.796, (2.642 sec/step)
step 11590 - trloss = 3.812, (2.640 sec/step)
step 11591 - trloss = 3.461, (2.640 sec/step)
step 11592 - trloss = 3.898, (2.640 sec/step)
step 11593 - trloss = 3.690, (2.640 sec/step)
step 11594 - trloss = 3.568, (2.640 sec/step)
step 11595 - trloss = 3.589, (2.640 sec/step)
step 11596 - trloss = 3.716, (2.641 sec/step)
step 11597 - trloss = 3.404, (2.641 sec/step)
step 11598 - trloss = 3.485, (2.641 sec/step)
step 11599 - trloss = 3.747, (2.641 sec/step)
step 11600 - trloss = 3.640, (2.640 sec/step)
stored done
validateLoss = 6.073, (6.851 sec/step)
step 11601 - trloss = 3.539, 

step 11755 - trloss = 3.663, (2.642 sec/step)
step 11756 - trloss = 3.494, (2.641 sec/step)
step 11757 - trloss = 3.887, (2.642 sec/step)
step 11758 - trloss = 3.575, (2.643 sec/step)
step 11759 - trloss = 3.686, (2.642 sec/step)
step 11760 - trloss = 3.835, (2.642 sec/step)
step 11761 - trloss = 3.657, (2.643 sec/step)
step 11762 - trloss = 3.496, (2.642 sec/step)
step 11763 - trloss = 3.881, (2.640 sec/step)
step 11764 - trloss = 3.837, (2.640 sec/step)
step 11765 - trloss = 3.687, (2.641 sec/step)
step 11766 - trloss = 3.934, (2.641 sec/step)
step 11767 - trloss = 3.513, (2.641 sec/step)
step 11768 - trloss = 3.835, (2.639 sec/step)
step 11769 - trloss = 3.848, (2.640 sec/step)
step 11770 - trloss = 3.565, (2.640 sec/step)
step 11771 - trloss = 3.627, (2.639 sec/step)
step 11772 - trloss = 3.380, (2.639 sec/step)
step 11773 - trloss = 3.979, (2.638 sec/step)
step 11774 - trloss = 3.751, (2.639 sec/step)
step 11775 - trloss = 3.665, (2.639 sec/step)
step 11776 - trloss = 3.673, (2.64

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%max_const_size, 'utf-8')
                
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())